In [1]:
pip install pyforest

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14606 sha256=6541a47ea5d0f0b5d397131889ef18eac7d24ae385455e034a1347311328e806
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\d5\1a\3e\6193fe1c56168f5df4aef57d8411033ba4611881135d495727
Successfully built pyforest
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyforest import *
lazy_imports()

['from sklearn.preprocessing import MinMaxScaler',
 'from pyspark import SparkContext',
 'import fastai',
 'import awswrangler as wr',
 'import altair as alt',
 'import os',
 'import spacy',
 'import xgboost as xgb',
 'from sklearn import svm',
 'from sklearn.linear_model import Lasso',
 'import seaborn as sns',
 'import numpy as np',
 'from sklearn.linear_model import LogisticRegression',
 'from sklearn.feature_extraction.text import TfidfVectorizer',
 'from sklearn.linear_model import LassoCV',
 'from sklearn.preprocessing import StandardScaler',
 'import sys',
 'from dask import dataframe as dd',
 'from PIL import Image',
 'from sklearn.impute import SimpleImputer',
 'from sklearn.ensemble import GradientBoostingClassifier',
 'from sklearn.preprocessing import OneHotEncoder',
 'import cv2',
 'from sklearn.model_selection import GridSearchCV',
 'from sklearn.model_selection import RandomizedSearchCV',
 'from sklearn.ensemble import RandomForestClassifier',
 'import imutils',
 'import

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
train_data=pd.read_csv('bigmart_train.csv')
test_data=pd.read_csv('bigmart_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>